# 综述
TensorFlow 提供了两种方法来控制随机数生成过程：

- 通过明确使用 tf.random.Generator 对象。每个此类对象都会在 tf.Variable 中维护一个状态，该状态在每次生成随机数后都会发生改变。

- 通过使用纯函数式无状态随机函数，如 tf.random.stateless_uniform。在同一设备上调用具有相同参数（包括种子）的这些函数会产生相同的结果。

> 警告：目前尚未弃用 TF 1.x 中的旧版 RNG（如 tf.random.uniform 和 tf.random.normal），但强烈建议不要使用。
> 
> 警告：不保证随机数在不同 TensorFlow 版本间一致，请参阅：版本兼容性

In [1]:
import tensorflow as tf

# Creates 2 virtual devices cpu:0 and cpu:1 for using distribution strategy
physical_devices = tf.config.experimental.list_physical_devices("CPU")
tf.config.experimental.set_virtual_device_configuration(
    physical_devices[0], [
        tf.config.experimental.VirtualDeviceConfiguration(),
        tf.config.experimental.VirtualDeviceConfiguration()
    ])

# tf.random.Generator 类
要点：
- 每次调用结果不一样
- 通过tf.Variable维护一个内部状态（因为是Variable变量，可以被checkpoint，自动控制依赖项，线程安全）

如何创建：
- 通过手动创建 tf.random.Generator类的一个对象，可以获得该生成器
- 调用 tf.random.get_global_generator()，您可以获得默认全局生成器：


# 创建方法1：tf.random.Generator
tf.random.Generator.from_seed,种子是任何非负整数，alg表示使用的RNG算法

In [3]:
g1 = tf.random.Generator.from_seed(1, alg='philox')
print(g1.normal(shape=[2, 3]))

tf.Tensor(
[[ 0.43842274 -0.53439844 -0.07710262]
 [ 1.5658046  -0.1012345  -0.2744976 ]], shape=(2, 3), dtype=float32)


Generator.from_non_deterministic_state：生成器首先会处于非确定状态，具体取决于时间和操作系统等因素。

In [4]:
g = tf.random.Generator.from_non_deterministic_state()
print(g.normal(shape=[2, 3]))

tf.Tensor(
[[-1.0283433   0.5518764   1.2624873 ]
 [ 0.98823726  0.32675055  1.5768538 ]], shape=(2, 3), dtype=float32)


还有其他方法可以创建生成器，比如说通过显式状态创建@TODO

# 创建方法2：tf.random.get_global_generator

获取全局生成器。
`第一次调用时会从非确定状态创建全局生成器，并放置在调用处的作用域的默认设备上`

比如说在GPU上创建，如果稍后在CPU使用，则会从GPU复制到CPU

In [10]:
g2 = tf.random.get_global_generator()
print(g2.normal(shape=[2, 3]))
tf.device("gpu")

tf.Tensor(
[[ 1.8368195  -0.49554613  1.2101773 ]
 [ 1.7602633   0.7141847  -0.62642   ]], shape=(2, 3), dtype=float32)


# 创建独立的随机数流
多个应用需要独立的随机数时，不想跨设备复制随机数生成器都要求能创建独立的随机数流。

通过使用 Generator.split 创建多个一定相互独立的生成器即可;与 normal 之类的 RNG 方法类似，split 会改变调用它的生成器的状态（上例中为 g）。除相互之间保持独立外，新生成器 (new_gs) 还一定独立于旧生成器 (g)。

In [11]:
g = tf.random.Generator.from_seed(1)
print(g.normal([]))
new_gs = g.split(3)
for new_g in new_gs:
  print(new_g.normal([]))
print(g.normal([]))

tf.Tensor(0.43842274, shape=(), dtype=float32)
tf.Tensor(2.536413, shape=(), dtype=float32)
tf.Tensor(0.33186463, shape=(), dtype=float32)
tf.Tensor(-0.07144657, shape=(), dtype=float32)
tf.Tensor(-0.79253083, shape=(), dtype=float32)


In [12]:
with tf.device("cpu"):  # change "cpu" to the device you want
  g = tf.random.get_global_generator().split(1)[0]  
  print(g.normal([]))  # use of g won't cause cross-device copy, unlike the global generator

tf.Tensor(1.0648588, shape=(), dtype=float32)


注：在理论上，此处可以使用 from_seed（而不是 split）之类的构造函数获取新生成器，但这样做无法保证新生成器与全局生成器相互独立。同时也有使用同一种子或导致产生重叠随机数流的种子意外创建两个生成器的风险。

# 与tf.function交互
与 tf.function 一起使用时，tf.random.Generator 遵循与 tf.Variable 相同的原则。这包括三个方面：
1. 在外部创建：调用该函数时，用户需要确保生成器对象仍处于活动状态（没有被回收）
2. 在内部创建：只有 tf.function 第一次运行时，才可以在其内部创建生成器
3. 作为参数传递：具有不同状态大小的不同生成器对象则会导致回溯。

In [13]:
# 外部
g = tf.random.Generator.from_seed(1)
@tf.function
def foo():
  return g.normal([])
print(foo())

tf.Tensor(0.43842274, shape=(), dtype=float32)


In [14]:
# 内部
g = None
@tf.function
def foo():
  global g
  if g is None:
    g = tf.random.Generator.from_seed(1)
  return g.normal([])
print(foo())
print(foo())

tf.Tensor(0.43842274, shape=(), dtype=float32)
tf.Tensor(1.6272374, shape=(), dtype=float32)


In [15]:
# 传参
num_traces = 0
@tf.function
def foo(g):
  global num_traces
  num_traces += 1
  return g.normal([])
foo(tf.random.Generator.from_seed(1))
foo(tf.random.Generator.from_seed(2))
print(num_traces)

1


# 与分布策略交互
也包括三种：
1. 在分布策略的外部创建生成器，会被序列化访问此生成器不同副本，没一个都是不同的随机数

> 会有性能问题，因为生成器的设备与副本不同

In [16]:
g = tf.random.Generator.from_seed(1)
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  def f():
    print(g.normal([]))
  results = strat.run(f)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1')
tf.Tensor(0.43842274, shape=(), dtype=float32)
tf.Tensor(1.6272374, shape=(), dtype=float32)


2. 在策略内部创建生成器

> 不允许在策略作用域内部创建生成器，因为这会导致在如何复制生成器方面出现歧义。比方说，是应该复制生成器，从而让每一个副本都获得相同的随机数，还是应该“拆分”，从而让每一个副本获得不同的随机数。


ps:请注意，Strategy.run 会在策略作用域内隐式运行参数函数：

In [17]:
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
with strat.scope():
  try:
    tf.random.Generator.from_seed(1)
  except ValueError as e:
    print("ValueError:", e)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1')
ValueError: Creating a generator within a strategy scope is disallowed, because there is ambiguity on how to replicate a generator (e.g. should it be copied so that each replica gets the same random numbers, or 'split' so that each replica gets different random numbers).


In [18]:
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
def f():
  tf.random.Generator.from_seed(1)
try:
  strat.run(f)
except ValueError as e:
  print("ValueError:", e)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1')
INFO:tensorflow:Error reported to Coordinator: Creating a generator within a strategy scope is disallowed, because there is ambiguity on how to replicate a generator (e.g. should it be copied so that each replica gets the same random numbers, or 'split' so that each replica gets different random numbers).
Traceback (most recent call last):
  File "/home/hdw/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/coordinator.py", line 297, in stop_on_exception
    yield
  File "/home/hdw/anaconda3/lib/python3.7/site-packages/tensorflow/python/distribute/mirrored_run.py", line 323, in run
    self.main_result = self.main_fn(*self.main_args, **self.main_kwargs)
  File "/home/hdw/anaconda3/lib/python3.7/site-packages/tensorflow/python/autograph/impl/api.py", line 275, in wrapper
    return func(*args, **kwargs)
  File "<ipython-input-

3. 将生成器作为参数传递给 Strategy.run

In [19]:
strat = tf.distribute.MirroredStrategy(devices=["cpu:0", "cpu:1"])
gs = tf.random.get_global_generator().split(2)
# to_args is a workaround for the absence of APIs to create arguments for 
# run. It will be replaced when such APIs are available.
def to_args(gs):  
  with strat.scope():
    def f():
      return [gs[tf.distribute.get_replica_context().replica_id_in_sync_group]]
    return strat.run(f)
args = to_args(gs)
def f(g):
  print(g.normal([]))
results = strat.run(f, args=args)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1')
tf.Tensor(1.3995249, shape=(), dtype=float32)
tf.Tensor(-0.18214026, shape=(), dtype=float32)


# 无状态RNG

无状态 RNG 的使用方法非常简单。因为它们是纯函数，不涉及状态或副作用。

> 但是相同的seed，会导致相同结果

In [20]:
print(tf.random.stateless_normal(shape=[2, 3], seed=[1, 2]))
print(tf.random.stateless_normal(shape=[2, 3], seed=[1, 2]))

tf.Tensor(
[[ 0.5441101   0.20738031  0.07356433]
 [ 0.04643455 -1.3015898  -0.95385665]], shape=(2, 3), dtype=float32)
tf.Tensor(
[[ 0.5441101   0.20738031  0.07356433]
 [ 0.04643455 -1.3015898  -0.95385665]], shape=(2, 3), dtype=float32)


# 算法
philox

XLA设备还包括：threefry

@TODO